# Logistic regression model

## Recreating the logisitic regression from the paper using Sci-kit learn

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import string as str
%matplotlib inline

In [50]:
#import data
import pandas as pd

#Read in dataset with all rows included
df = pd.read_sas("/Users/roddy.jacques/Documents/DSA Project/Data/alldata3.sas7bdat")

#6931 DBD apps
dbd_apps = df[(df["eli_DBD"]==1)&(df["FAMILY_APPROACHED"]==2)]

#6060 DBD apps to match cohort in pape
dbd_apps = dbd_apps[(dbd_apps["eth_grp"]!=5)&(dbd_apps["FORMAL_APR_WHEN"]!=4)&(dbd_apps["donation_mentioned"]!=-1)
                    &(dbd_apps["FAMILY_WITNESS_BSDT"]!=9)&(dbd_apps["GENDER"]!=9)]
     
#9965 DCD apps
dcd_apps = df[(df["eli_DCD"]==1)&(df["FAMILY_APPROACHED"]==2)]

#9405 DCD apps to match cohort in paper
dcd_apps = dcd_apps[(dcd_apps["GENDER"]!=9)&(dcd_apps["cod_neuro"].notna())&(dcd_apps["eth_grp"]!=5)&(dcd_apps["donation_mentioned"]!=-1)&
                    (~dcd_apps["DTC_WD_TRTMENT_PRESENT"].isin([8,9]))]

/Users/roddy.jacques/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sas/sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")
/Users/roddy.jacques/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sas/sas7bdat.py:809: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._string_chunk[js, :]


In [66]:
# Columns used to create DBD model in paper
dbd_cols = ["wish", "FORMAL_APR_WHEN", "donation_mentioned", "app_nature", "eth_grp", "religion_grp", "GENDER", "FAMILY_WITNESS_BSDT", "DTC_PRESENT_BSD_CONV", 
            "acorn_new", "adult","FAMILY_CONSENT"]

dbd_apps[dbd_cols].astype(int)

dbd_model_data = dbd_apps[dbd_cols]
dbd_model_data2 = pd.get_dummies(data=dbd_model_data,columns=dbd_cols[:-1])

dbd_features = dbd_model_data2.drop("FAMILY_CONSENT",axis=1)
dbd_consents = dbd_model_data2["FAMILY_CONSENT"]

dbd_feature_names = dbd_features.columns.tolist()

In [73]:
LR_model = LogisticRegression(penalty='none')

LR_model.fit(dbd_features,dbd_consents)

odds_ratios = np.exp(LR_model.coef_) 

var_names = np.empty(39,dtype="str")

for i in range(39):
    var_names[i] = dbd_feature_names[i].rsplit("_",1)
    
var_names

ValueError: setting an array element with a sequence

In [47]:
cols_and_ors = np.column_stack([dbd_features.columns.tolist(),odds_ratios[0]])
cols_and_ors

array([['wish_1.0', '0.19286004024379008'],
       ['wish_2.0', '4.591863674646195'],
       ['wish_3.0', '1.4613286364010425'],
       ['wish_4.0', '3.563264061358002'],
       ['wish_5.0', '0.29824758358901193'],
       ['FORMAL_APR_WHEN_1.0', '1.9935143421525394'],
       ['FORMAL_APR_WHEN_2.0', '0.8658885020778523'],
       ['FORMAL_APR_WHEN_3.0', '0.7967512215531105'],
       ['donation_mentioned_1.0', '0.7433358679125524'],
       ['donation_mentioned_2.0', '0.9356188221607649'],
       ['donation_mentioned_3.0', '1.3229661935567882'],
       ['donation_mentioned_4.0', '1.4947593870990241'],
       ['app_nature_1.0', '1.7930829135508939'],
       ['app_nature_2.0', '1.5715218897351706'],
       ['app_nature_3.0', '0.488071297547716'],
       ['eth_grp_1.0', '1.837073139892467'],
       ['eth_grp_2.0', '0.9152575645804563'],
       ['eth_grp_3.0', '0.5312677738633718'],
       ['eth_grp_4.0', '1.5396458229142043'],
       ['religion_grp_1.0', '1.495432715849936'],
       ['religio